# Menginstall Dependency

In [ ]:
!pip install "pyarrow==19.0.0" --force-reinstall -q
!pip install -U datasets bitsandbytes sentencepiece -q
!pip install -U "transformers>=4.44.0" "accelerate>=0.34.0" "peft>=0.11.0" -q
!pip install python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.0 which is incompatible.


# Import Library

In [ ]:
import os
import glob
import torch
from google.colab import drive
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model

# Mount Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)
LOCAL_DATA_DIR = "/content/drive/MyDrive/Data_Training/Semua"

Mounted at /content/drive


# Konfigurasi Global & Pemuatan Data

In [ ]:
MODEL_NAME = "Qwen/Qwen3-8B"
BLOCK_SIZE = 1024

all_files = glob.glob(os.path.join(LOCAL_DATA_DIR, "*.txt"))
print(f"Total file.txt yang ditemukan: {len(all_files)}")

if not all_files:
    raise ValueError(f"Tidak ada file.txt yang ditemukan di {LOCAL_DATA_DIR}. Periksa path dan proses unzip.")

raw_dataset = load_dataset("text", data_files=all_files, split="train")

print(f"Jumlah baris sebelum cleaning: {len(raw_dataset)}")
raw_dataset = raw_dataset.filter(lambda x: x['text'] is not None and x['text'].strip() != '')
print(f"Jumlah baris setelah cleaning: {len(raw_dataset)}")

Total file.txt yang ditemukan: 1
Jumlah baris sebelum cleaning: 82387


Filter:   0%|          | 0/82387 [00:00<?, ? examples/s]

Jumlah baris setelah cleaning: 41204


# Split Dataset

In [ ]:
print("Memisahkan dataset 90:10...")
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})
print(dataset)

Memisahkan dataset 90:10...
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 37083
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 4121
    })
})


In [ ]:
DatasetDict({
    'train': dataset['train']['text'],
    'validation': dataset['validation']
})

DatasetDict({
    train: Column(['Marko Simic kembali gabung ke Persija Jakarta dan diumumkan pada Selasa (20/6/2023). Dikontrak dua tahun, kehadiran pemain 3 tahun itu diharapkan kembali membuat lini depan Persija jadi semakin tajam. Perekrutan Simic dinilai sesuai kebutuhan. Sehingga, Persija tak sepakat jika perekrutan Simic adalah langkah panic buying alias pembelian panik karena Persija tak kunjung mendatangkan pemain asing lagi pasca perekrutan Ryo Matsumura. Dikutip dari detikSport, pergerakan Persija Jakarta terhitung lambat untuk memenuhi kuota enam pemain asing yang diizinkan di Liga 1 2023/2024. Simic baru pemain asing ketiga yang mengisi slot itu, sebelumnya baru ada Ondrej Kudela dan Ryo Matsumura. "Kalau dibilang panic buying, ya nggak lah. Jika panic buying, pasti semuanya maunya cepat-cepat. Kami berusaha melakukan ini secara rasional dan terukur dengan mempertimbangkan banyak hal," kata Ganesha. "Kami menganggap Simic bisa membantu meraih target yang kami inginkan dan 

# Tokenisasi dan Chunking

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

BLOCK_SIZE = 1024
STRIDE = 256

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_with_sliding_window(examples):
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=BLOCK_SIZE,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_length=True,
        padding="max_length"
    )

    sample_map = outputs.pop("overflow_to_sample_mapping")

    labels = outputs["input_ids"].copy()

    labels = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels
    ]

    outputs["labels"] = labels
    return outputs

print("=== Melakukan Tokenisasi dengan Sliding Window (Overlap) ===")

lm_datasets = dataset.map(
    tokenize_with_sliding_window,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names
)

print(f"Dataset final dengan overlap: {lm_datasets}")

=== Melakukan Tokenisasi dengan Sliding Window (Overlap) ===


Map (num_proc=4):   0%|          | 0/37083 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4121 [00:00<?, ? examples/s]

Dataset final dengan overlap: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 70511
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 7930
    })
})


# Visualisasi Data Untuk Training

In [ ]:
for i in range(5):
  print('Block size     : ',len(lm_datasets['train']['input_ids'][i]))
  print('Input ids      : ',lm_datasets['train']['input_ids'][i])
  print('Attention Mask : ',lm_datasets['train']['attention_mask'][i])
  print('Labels         : ',lm_datasets['train']['labels'][i])
  print('Contoh teks:')
  display(tokenizer.decode(lm_datasets['train'][i]['input_ids'], skip_special_tokens=True))

  print('\n')

Block size     :  1024
Input ids      :  [8949, 78, 4467, 292, 595, 68705, 55879, 2185, 1962, 20465, 28130, 63033, 9101, 1853, 372, 372, 8656, 31811, 23204, 15428, 320, 17, 15, 14, 21, 14, 17, 15, 17, 18, 568, 422, 61614, 70019, 96821, 57523, 11, 1962, 31245, 54690, 54184, 466, 220, 18, 57523, 35691, 1853, 12982, 391, 8656, 595, 68705, 76234, 326, 6591, 409, 848, 20465, 28130, 502, 2767, 5234, 42230, 259, 1630, 309, 13, 72478, 9855, 48524, 4467, 292, 11623, 321, 2143, 15537, 84695, 1962, 8088, 69886, 13, 1345, 71, 65401, 11, 20465, 28130, 18116, 21017, 585, 266, 70879, 281, 485, 9855, 48524, 4467, 292, 37560, 8688, 83502, 21975, 11833, 15534, 63567, 69564, 7215, 1579, 59469, 20465, 28130, 18116, 23330, 73, 2185, 44776, 266, 86012, 54184, 466, 438, 287, 73944, 6368, 924, 281, 485, 9855, 48524, 431, 16032, 6867, 1242, 5690, 13, 422, 88994, 573, 24200, 3392, 1579, 58737, 11, 817, 1389, 19262, 20465, 28130, 63033, 1982, 22492, 2185, 38306, 266, 19593, 1833, 1481, 6023, 36600, 6089, 81852, 

'Marko Simic kembali gabung ke Persija Jakarta dan diumumkan pada Selasa (20/6/2023). Dikontrak dua tahun, kehadiran pemain 3 tahun itu diharapkan kembali membuat lini depan Persija jadi semakin tajam. Perekrutan Simic dinilai sesuai kebutuhan. Sehingga, Persija tak sepakat jika perekrutan Simic adalah langkah panic buying alias pembelian panik karena Persija tak kunjung mendatangkan pemain asing lagi pasca perekrutan Ryo Matsumura. Dikutip dari detikSport, pergerakan Persija Jakarta terhitung lambat untuk memenuhi kuota enam pemain asing yang diizinkan di Liga 1 2023/2024. Simic baru pemain asing ketiga yang mengisi slot itu, sebelumnya baru ada Ondrej Kudela dan Ryo Matsumura. "Kalau dibilang panic buying, ya nggak lah. Jika panic buying, pasti semuanya maunya cepat-cepat. Kami berusaha melakukan ini secara rasional dan terukur dengan mempertimbangkan banyak hal," kata Ganesha. "Kami menganggap Simic bisa membantu meraih target yang kami inginkan dan saya pikir baik untuk klub dan Si



Block size     :  1024
Input ids      :  [50, 1225, 90786, 1853, 23968, 1521, 288, 585, 76234, 68886, 36686, 329, 10386, 10371, 10581, 484, 81490, 31954, 6850, 817, 1241, 51188, 11, 54425, 355, 360, 47656, 355, 44821, 12540, 5419, 72, 511, 84168, 8146, 307, 1853, 92800, 451, 1878, 72, 220, 16, 19, 22, 63033, 1051, 8656, 326, 36986, 13, 26692, 13229, 44432, 484, 51188, 1527, 585, 23968, 320, 42, 8041, 40, 8, 54425, 3065, 332, 11, 511, 62402, 90721, 408, 1579, 69927, 57523, 220, 17, 15, 16, 20, 92064, 45960, 70, 22051, 9101, 13294, 18633, 276, 730, 1225, 9247, 276, 1853, 13177, 10386, 393, 50403, 22222, 827, 74286, 11, 16923, 21226, 43834, 90786, 18116, 71136, 68886, 55766, 13, 328, 1114, 5059, 8251, 16225, 512, 3096, 12466, 35691, 31811, 220, 17, 15, 16, 23, 54425, 3065, 332, 11, 31954, 6850, 817, 1241, 51188, 1853, 43834, 90786, 51518, 391, 2143, 220, 16, 15, 22, 83255, 13, 37542, 266, 1800, 74, 6283, 3651, 416, 63033, 9354, 324, 11, 30572, 26095, 472, 722, 393, 399, 78, 11, 19729, 4

'Sekolah di Indonesia didesak membuat sistem pengaduan yang melindungi korban perundungan, menyusul kasus bunuh diri seorang murid di SMP Negeri 147 Jakarta pekan lalu. Komisi Perlindungan Anak Indonesia (KPAI) menyebut, sejak Permendikbud tahun 2015 tentang Pencegahan dan Penanganan Kekerasan di Satuan Pendidikan disahkan, mayoritas sekolah tak memiliki sistem tersebut. Sementara catatan lembaga itu pada 2018 menyebut, korban perundungan di sekolah mencapai 107 anak. Kasat Reskrim Polres Jakarta Timur, AKBP Hery Purno, mengatakan hingga saat ini polisi telah memeriksa pihak sekolah terkait meninggalnya murid berinisal N. Mereka yang diperiksa di antaranya kepala sekolah, wakil kepala sekolah, dan wali kelas. Kendati demikian, pemeriksaan yang sudah berlangsung sejak Jumat pekan lalu itu, belum menemukan titik terang. "Kita masih dalami, nanti kalau sudah semua kita dapatkan keterangan, nanti akan kita sampaikan," ujar Hery Purnomo kepada Quin Pasaribu yang melaporkan untuk BBC News In



Block size     :  1024
Input ids      :  [97659, 595, 2367, 9198, 276, 296, 91280, 993, 266, 30, 393, 4204, 294, 46447, 12589, 305, 65401, 259, 365, 300, 59469, 13112, 324, 1853, 19868, 72876, 4598, 2185, 328, 579, 337, 6743, 1365, 5232, 76427, 33598, 6321, 9243, 72876, 69837, 511, 13718, 300, 13995, 1466, 18668, 329, 1466, 30, 16358, 29628, 24200, 393, 6132, 61892, 41068, 17106, 23204, 466, 281, 6996, 585, 43834, 90786, 9101, 48228, 84, 17922, 11, 1471, 13229, 42369, 32670, 1833, 33469, 595, 53311, 1562, 276, 12, 870, 276, 451, 13, 330, 45, 15359, 1562, 276, 12, 870, 276, 10371, 8310, 7755, 595, 1315, 276, 38372, 10207, 26682, 51188, 20414, 10371, 58554, 524, 74, 48524, 11, 3267, 72, 32670, 54211, 28337, 2143, 595, 53311, 5234, 84, 24074, 1189, 37542, 355, 55766, 1982, 92046, 31811, 23204, 15428, 320, 16, 19, 14, 15, 16, 8, 326, 36986, 13, 85537, 321, 328, 7876, 300, 9101, 19858, 300, 92800, 451, 220, 16, 19, 22, 63033, 11, 32559, 4711, 1881, 11, 19729, 459, 8656, 511, 84168, 8146, 

' hak kesehatan masyarakat adat? Pria dianiaya hingga tewas karena tidur di Masjid Agung Sibolga – Apakah fungsi masjid hanya sebatas rumah ibadah? Akhir dari Paling banyak dibaca Selain pihak sekolah dan keluarga, polisi juga akan meminta keterangan teman-teman N. "Nanti teman-teman yang punya kaitan atau berhubungan dengan yang bersangkutan, pasti akan kita mintai keterangan semuanya." Kasus tersebut terjadi pada Selasa (14/01) lalu. Wakil Sarpas dan Humas SMP N 147 Jakarta, Misnetty, mengatakan seorang muridnya berinisal N meninggal setelah lompat dari lantai empat gedung sekolahnya. Saat itu, ia berada berada di sekolah dan mendengar suara seseorang yang jatuh. "Kemudian saya mendengar ada teriakan-teriakan karena saya tidak melihat langsung. Lalu saya buka pintu dan keluar ada siswa yang jatuh," ujar Misnetty kepada Kompas.com. Korban lantas di bawa ke salah satu klinik terdekat dan kemudian dirujuk ke Rumah Sakit Tugu Ibu. Tapi karena keterbatasan alat, korban dipindah ke Rumah S



Block size     :  1024
Input ids      :  [1225, 9247, 276, 7767, 50858, 74, 51188, 13177, 10386, 393, 50403, 22222, 13, 328, 66421, 10371, 5103, 9810, 661, 35691, 10207, 45831, 6792, 281, 763, 70, 22051, 10371, 1982, 3741, 72, 24200, 64184, 6053, 43834, 90786, 11, 817, 86, 585, 89359, 59426, 11, 52315, 9991, 11, 9101, 49607, 63884, 13, 7767, 817, 2628, 276, 35691, 42369, 827, 3065, 332, 8656, 43834, 90786, 289, 1630, 579, 1833, 300, 524, 281, 5359, 10962, 28618, 36686, 329, 10386, 259, 484, 585, 1962, 7052, 45388, 10371, 26183, 1466, 1853, 585, 9275, 51316, 52315, 9991, 11, 49607, 63884, 11, 38372, 59426, 13, 393, 5359, 10962, 28618, 35691, 1833, 54182, 9662, 269, 7963, 618, 9101, 452, 41911, 2551, 13, 350, 2068, 59469, 31281, 72, 2584, 276, 6792, 281, 763, 70, 22051, 11, 2953, 324, 332, 10392, 2152, 11, 27281, 34234, 52315, 9991, 10371, 10207, 5559, 10581, 391, 669, 276, 47656, 355, 817, 1241, 51188, 13, 330, 50, 1225, 90786, 18116, 8310, 7755, 68886, 36686, 329, 10386, 91665, 65401

'ekerasan Di Lingkungan Satuan Pendidikan. Sistem yang dimaksud itu berupa tim pencegahan yang terdiri dari kepala sekolah, perwakilan guru, siswa, dan orang tua. Di peraturan itu juga disebutkan sekolah wajib memasang papan layanan pengaduan tindak kekerasan yang mudah diakses oleh siswa, orang tua, atau guru. Papan layanan itu memuat nomor telepon dan alamat email. Tapi karena ketiadaan tim pencegahan, menurut Retno, tidak ada siswa yang berani melaporkan kasus perundungan. "Sekolah tak punya sistem pengaduan sehingga mereka enggan mengadu dan melapor. Kalau mengadu atau melapor, ya sama saja bunuh diri atau sama dengan mencari masalah baru. Sudah dibully, mengadu, jadi masalah." "Guru-guru juga sebenarnya sih saya yakin tahu bahwa ada pembully-an di sekolah, hanya kepekaan orang dewasa di sekitar anak yang kurang." "Jadi memang mencari informasi yang tepat ke anak, syaratnya anak percaya sama kita (guru). Ini yang kurang dilakukan. Jadi informasi detail soal bully, bertanya langsung



Block size     :  1024
Input ids      :  [28205, 88, 307, 1579, 3719, 17752, 33611, 23900, 52418, 416, 9855, 5742, 355, 393, 813, 64, 20957, 619, 12589, 10207, 1389, 585, 62090, 266, 2953, 524, 5559, 47656, 355, 5750, 9011, 2257, 276, 20153, 268, 10371, 10207, 13229, 43463, 585, 5359, 10371, 1521, 36960, 10581, 579, 266, 8656, 54184, 6664, 258, 434, 1893, 431, 449, 645, 80, 1417, 6996, 370, 9101, 87630, 524, 4554, 294, 36960, 276, 50916, 277, 79247, 4360, 472, 810, 258, 13, 36866, 268, 55766, 57635, 50537, 2257, 1853, 84938, 2444, 355, 9101, 3687, 60877, 530, 511, 62402, 55883, 8717, 11, 220, 17, 24, 4350, 84, 2780, 220, 17, 15, 16, 22, 13, 40436, 307, 19858, 300, 393, 813, 64, 20957, 619, 12589, 730, 2855, 288, 20605, 268, 2340, 370, 57715, 1644, 3346, 27273, 86, 10148, 19729, 459, 8656, 11, 47656, 355, 16806, 18212, 3029, 96368, 1853, 2185, 38834, 6792, 3272, 299, 742, 274, 8629, 13, 36866, 268, 55766, 10207, 13229, 259, 51628, 276, 320, 45070, 8, 43463, 585, 5359, 9101, 9015, 2257

'Penyidik Subdit Cyber Crime Ditreskrimsus Polda Metro Jaya bergerak cepat menangani kasus penyebaran konten yang berisi percakapan yang diduga melibatkan pemimpin FPI Rizieq Shihab dan tersangka dugaan makar Firza Husein. Konten tersebut telah tersebar di beberapa situs dan media sosial sejak Minggu, 29 Januari 2017. Kabid Humas Polda Metro Jaya Kombes Raden Prabowo Argo Yuwono mengatakan, kasus ini pertama kali diungkap tim patroli siber. Konten tersebut berisi tampilan (capture) percakapan dan gambar berbau pornografi yang diduga Habib Rizieq alias HR dan Firza Husein alias F. "Tentunya kita punya cyber patrol. Setelah adanya cyber patrol, kita menemukan beberapa akun yang diduga ada gambar pornografi, yang ada gambar di situ diduga adalah HR dan F," ujar Argo di Mapolda Metro Jaya, Senin (30/1/2017). Hasil patroli siber yang dilakukan tim penyidik Polda Metro Jaya itu kemudian dilaporkan ke pimpinan. Selanjutnya pimpinan menginstruksikan agar dilakukan penyelidikan pada kasus penye

# Inisialisasi

# QLoRA Config

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

## Load Base Model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print(base_model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06

## Konfigurasi Lora

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 10,911,744 || all params: 8,201,647,104 || trainable%: 0.1330


## Data Collator

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Metrics Callback

## Callback untuk menyimpan semua metrik (train/eval, loss/perplexity)

In [ ]:
import math
import wandb
from transformers import TrainerCallback

class WandBPerplexityCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.metrics = {
            'train_loss': [],
            'train_perplexity': [],
            'eval_loss': [],
            'eval_perplexity': []
        }

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Dipanggil setiap 'logging_steps' (saat training)"""
        if logs and 'loss' in logs:
            step = state.global_step
            loss = logs['loss']

            try:
                ppl = math.exp(loss)
            except OverflowError:
                ppl = float('inf')

            wandb.log({"train/perplexity": ppl})

            self.metrics['train_loss'].append((step, loss))
            self.metrics['train_perplexity'].append((step, ppl))

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Dipanggil setiap 'eval_steps' (saat validasi)"""
        if metrics and 'eval_loss' in metrics:
            step = state.global_step
            loss = metrics['eval_loss']

            try:
                ppl = math.exp(loss)
            except OverflowError:
                ppl = float('inf')

            wandb.log({"eval/perplexity": ppl})

            self.metrics['eval_loss'].append((step, loss))
            self.metrics['eval_perplexity'].append((step, ppl))

metrics_callback = WandBPerplexityCallback()

# Setup Up Monitoring WANDB

In [ ]:
import wandb
from dotenv import load_dotenv

env_path = '/content/drive/MyDrive/secrets/.env'
load_dotenv(env_path)
wandb_key = os.getenv('WANDB_API_KEY')
wandb.login(key=wandb_key)
wandb.init(project="CPT Tim 1", entity="Tim-1", name="Qwen3-8B-All_Data")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# Setup untuk Trainer

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Qwen3-All_Data_cpt_model_checkpoints"
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,  # Training skala besar  : 4
    per_device_eval_batch_size=4,   # Training skala besar  : 4
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=2000,                 # Training skala besar  : 2000
    logging_steps=50,               # Training skala besar  : 500
    save_steps=200,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=False,
    bf16=True,
    report_to="wandb",
    save_total_limit=1,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[metrics_callback],
)

/tmp/ipython-input-3168915875.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Pengecekan Checkpoint dan Memulai Continued Pre-Training

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
import os

last_checkpoint = None
if os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint:
    print(f"Checkpoint ditemukan di: {last_checkpoint}")
    print("Melanjutkan training dari langkah terakhir...")
else:
    print("Tidak ada checkpoint valid ditemukan. Memulai training dari awal...")

train_result = trainer.train(resume_from_checkpoint=last_checkpoint)

trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

Tidak ada checkpoint valid ditemukan. Memulai training dari awal...


Step,Training Loss,Validation Loss
2000,1.340000,1.346113
4000,1.346800,1.331875
6000,1.346400,1.325385


# Evaluate

In [ ]:
trainer.evaluate()

In [ ]:
import math

eval_loss = trainer.evaluate()['eval_loss']
perplexity = math.exp(eval_loss)

print(f"Perplexity: {perplexity}")

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

train_loss_steps = [data[0] for data in metrics_callback.metrics['train_loss']]
train_loss_values = [data[1] for data in metrics_callback.metrics['train_loss']]

eval_loss_steps = [data[0] for data in metrics_callback.metrics['eval_loss']]
eval_loss_values = [data[1] for data in metrics_callback.metrics['eval_loss']]

ax1.plot(train_loss_steps, train_loss_values, label='Training Loss', alpha=0.7)
ax1.plot(eval_loss_steps, eval_loss_values, label='Validation Loss', alpha=0.7)
ax1.set_xlabel('Steps')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True)

train_ppl_steps = [data[0] for data in metrics_callback.metrics['train_perplexity']]
train_ppl_values = [data[1] for data in metrics_callback.metrics['train_perplexity']]

eval_ppl_steps = [data[0] for data in metrics_callback.metrics['eval_perplexity']]
eval_ppl_values = [data[1] for data in metrics_callback.metrics['eval_perplexity']]

ax2.plot(train_ppl_steps, train_ppl_values, label='Training Perplexity', alpha=0.7)
ax2.plot(eval_ppl_steps, eval_ppl_values, label='Validation Perplexity', alpha=0.7)
ax2.set_xlabel('Steps')
ax2.set_ylabel('Perplexity')
ax2.set_title('Training & Validation Perplexity')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Save Model

In [ ]:
print("Pelatihan selesai. Menyimpan model final.")
trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))